In [2]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

import sqlite3
import itertools

In [3]:
con = sqlite3.connect('backtest_2020\pnl.db')

Single Startegy PNL - 
1. Startegy Description (eg. Confluence Threshold - 100 (4H Confluence), Take Profit = 5%, Stop Loss = 20%)
2. Statisitics Table (PnL, Ratios, Profit Factor, Start Date, End Date, Number of Trades)
3. Cash Visualization
4. Trades Visualizaion

In [20]:
def get_perf_filepath(thresh, take_profit, stop_loss):
    return f'backtest_2020/{thresh}_4h/perfs/TP{int(take_profit)}_SL{int(stop_loss)}_perf.pkl'

def print_pnl(row_num):
    
    # Statistics Table
    row = params_table.iloc[row_num, :]

    backtest_id = row['backtest_id']

    rt_stats_sql_query = f"SELECT * FROM roundtrip_stats WHERE backtest_id='{backtest_id}' and type='Long trades'"
    row_rtstats = pd.read_sql(rt_stats_sql_query, con=con)

    rt_columns = ['Total_profit',
                  'Profit_factor',
                  'Percent_profitable',
                  'Gross_profit',
                  'Gross_loss']

    returns_sql_query = f"SELECT * FROM returns WHERE backtest_id='{backtest_id}'"
    row_returns = pd.read_sql(returns_sql_query, con=con)
    returns_columns = ['End_date',
                       'Start_date',
                       'Total_months',
                       'annual_return',
                       'calmar_ratio',
                       'common_sense_ratio',
                       'omega_ratio',
                       'sharpe_ratio',
                       'sortino_ratio']

    drawdown_sql_query = f"SELECT * FROM drawdown_stats WHERE backtest_id='{backtest_id}'"
    row_drawdown = pd.read_sql(drawdown_sql_query, con=con)
    row_drawdown = row_drawdown.rename(columns={'duration':'duration (in days)'})

    print("#"*30)
    print()
    print(f"Backtest Ref: {row['backtest_id']}")
    print()
    print("1. Parameters - ")
    display(row[['confluence_4h', 'take_profit', 'stop_loss']].to_frame().T)
    print()
    print("2. Roundtrip Stats - ")
    display(row_rtstats[rt_columns])
    print()
    print("3. Returns - ")
    display(row_returns[returns_columns])
    print()
    print("4. Worst Drawdown Periods - ")
    display(row_drawdown.drop('backtest_id', axis=1))
    
    perf = pd.read_pickle(backtest_id)
    cash = perf.cash.resample('D').ohlc().close.reset_index()
    cash.columns = ['Date', 'Cash (USDT)']
    fig = px.line(cash, x="Date", y="Cash (USDT)", title='Cash')

    fig.update_layout(
        title=f"Cash with Take Profit = {row['take_profit']}% and Stop Loss = {row['stop_loss']}%",
        xaxis=dict(
            title='Date',
            gridcolor='white',
            gridwidth=2,
        ),
        yaxis=dict(
            title='Cash (USDT)',
            gridcolor='white',
            gridwidth=2,
        ),
        paper_bgcolor='rgb(243, 243, 243)',
        plot_bgcolor='rgb(243, 243, 243)',
        legend=dict(itemsizing='constant')
    )

    fig.show()
    
    btc_usdt_price = perf.price.resample('D').ohlc()['close'].reset_index()
    btc_usdt_price.columns = ['Date', 'BTC Price (USDT)']
    
    signals = pd.read_pickle(backtest_id.replace('perfs', 'signals').replace('_perf', '_signals'))
    ## Data Preparation
    actions_agg = signals.resample('D').agg({'action':['max', 'count']}).dropna()

    num_actions = actions_agg['action']
    num_actions.index = num_actions.index.tz_localize('UTC')
    num_actions.reset_index(inplace=True)
    num_actions.columns = ['Date', 'Action', 'Count']

    trade_dat = pd.merge(btc_usdt_price,
                         num_actions,
                         on='Date',
                         how='left')
    trade_dat.fillna(value={'Action' : 'Waiting', 'Count' : 0}, inplace=True)
    
    fig = go.Figure()
    my_map = {"Enter Long" : 'goldenrod',
              "Take Profit" : 'blue', 
              "Stop Loss" : 'red'}
    hover_text = []
    for index, drow in trade_dat.iterrows():
        hover_text.append(('Action: {action}<br>'+
                          'Count: {count}<br>'+
                          'BTC/USDT Price: ${price}<br>').format(action=drow['Action'],
                                                                 count=int(drow['Count']),
                                                                 price=round(drow['BTC Price (USDT)'], 2)))

    trade_dat.loc[:,'text'] = hover_text


    for action in ['Enter Long', 'Take Profit', 'Stop Loss']:
        fig.add_trace(go.Scatter(x=trade_dat[trade_dat['Action']==action].Date,
                                 y=trade_dat[trade_dat['Action']==action]['BTC Price (USDT)'],
                                mode='markers',
                                name=action, 
                                marker=dict(color=my_map[action], 
                                            size=trade_dat[trade_dat['Action']==action].Count+5,
                                            line_color='black',
                                            line_width=0.5),
                                text=trade_dat[trade_dat['Action']==action].text))

    fig.add_trace(go.Scatter(x=trade_dat['Date'],
                             y=trade_dat['BTC Price (USDT)'],
                             mode='lines',
                             name='BTC / USDT Price',
                             opacity=0.5))
    
    fig.update_layout(
        title=f"Trade Entries and Exits with Take Profit = {row['take_profit']}% and Stop Loss = {row['stop_loss']}%",
        xaxis=dict(
            title='Date',
            gridcolor='white',
            gridwidth=2,
        ),
        yaxis=dict(
            title='BTC/USDT Price',
            gridcolor='white',
            gridwidth=2,
        ),
        paper_bgcolor='rgb(243, 243, 243)',
        plot_bgcolor='rgb(243, 243, 243)',
        legend=dict(itemsizing='constant')
    )
    
    fig.show()

    print("#"*30)

In [21]:
# Startegy Description
threshold = 100
tp_list = [25, 20, 15, 10, 5]
sl_list = [2.5, 5, 10, 15]
tpsl_optimisation_params = itertools.product(tp_list, sl_list)

params_table = pd.DataFrame(list(tpsl_optimisation_params), columns=['take_profit', 'stop_loss'])

filepaths = [get_perf_filepath(threshold, row['take_profit'], row['stop_loss']) for i, row in params_table.iterrows()]
params_table['backtest_id'] = filepaths
params_table['confluence_4h'] = threshold

In [22]:
for i in range(len(params_table)):
    print(i+1)
    print_pnl(i)

1
##############################

Backtest Ref: backtest_2020/100_4h/perfs/TP25_SL2_perf.pkl

1. Parameters - 


,confluence_4h,take_profit,stop_loss
0,100,25,2.5



2. Roundtrip Stats - 


,Total_profit,Profit_factor,Percent_profitable,Gross_profit,Gross_loss
0,754.012462,3.992849,0.6,1005.950478,-251.938016



3. Returns - 


...


3. Returns - 


,End_date,Start_date,Total_months,annual_return,calmar_ratio,common_sense_ratio,omega_ratio,sharpe_ratio,sortino_ratio
0,2020-12-31,2020-01-01,12,0.051556,None,21.386489,None,23.706037,inf



4. Worst Drawdown Periods - 


,net_drawdown_in_%,peak_date,valley_date,recovery_date,duration (in days)


##############################
